In [1]:
import sys
import pandas as pd
import pickle
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

sys.path.insert(0, '/home/zhengly/PFmap/PFmap/data_procession')
from consine_distances import load_data,MinMaxScaleClip
from utils import Ontology
from erro import feature_to_csv



class Data_process():

    def __init__(self,protein_file,num,grid_file="/home/zhengly/promap/data/CAFA/data_grid.pkl",assess_file="/home/zhengly/promap/data/CAFA/row_asses.pkl"):
        '''
        protein_file 是生成的蛋白特征文件，
        split_file是需要包含所需信息的蛋白文件,
        save_file是生成的文件需要保存的位置,
        prosim_file是蛋白相似性比对的库
        grid_file，assess_file是使用的map的位置文件
        '''
        self.protein_file=protein_file
        self.grid_file=grid_file
        self.assess_file=assess_file
        self.num=num
        self.__data__()

    def __data__(self):
        proteins_f=feature_to_csv(self.protein_file)
        proteins_f.dropna(axis=0,inplace=True)
        feature_data=proteins_f.iloc[:,:self.num]
        mia_data=load_data("/data/promap/data/protein/cafa4.csv",1485)
        mia_data.columns=range(len(mia_data.columns))
        feature_data=(feature_data - mia_data.min()) / ((mia_data.max() - mia_data.min()) + 1e-8)
        self.feature_data=feature_data
        self.proteins=list(proteins_f.index)
        self.data_grid=pickle.load(open(self.grid_file,"rb"))
        self.row_asses=pickle.load(open(self.assess_file,"rb"))


    def calculate_feature(self,row_num,size):
        protein_all=[]
        feature_all=[]
        data_grid=self.data_grid
        row_asses=self.row_asses
        proteins=self.proteins
        feature_data=self.feature_data
        for i in range(len(proteins)):
            col_list=np.zeros(size)
            protein=proteins[i]
            row=0
            col=0
            protein_all.append(proteins[i])
            #构建promap特征
            for j in range(len(data_grid['x'])):
                channel=data_grid['subtype'][j]   
                feature_index=row_asses[j]
                row=j%row_num
                col=j//row_num
                col_list[col][row]=feature_data.iloc[i,feature_index]
            feature_all.append(col_list)
        data_t=[protein_all,feature_all]
        data_t=pd.DataFrame(data_t)
        data_t=data_t.T
        data_t.columns=['Proteins','Promap_feature']
        return data_t

2022-11-28 23:48:29.142239: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
# example
import pandas as pd
sys.path.insert(0,'/home/zhengly/PFmap/PFmap/plot')
from plt_sample import plotSampleGrid

data_sample=Data_process(protein_file='/home/zhengly/PFmap/example/output-protein.dat',num=1485)
data_s=data_sample.calculate_feature(row_num=39,size=(39,39))
data_feature=pd.DataFrame(data_s["Promap_feature"][0])
data_grid=pd.read_pickle("/home/zhengly/PFmap/PFmap/data/data_grid.pkl")
plotSampleGrid(data_feature,data_grid,"O14793","UMEP","/home/zhengly/PFmap/example/O14793")